In [1]:
import torch
import torch.nn.functional as F

torch.manual_seed(1)

# **1. 파이토치로 소프트맥스의 비용 함수 구현하기 (low-level)**
---

소프트맥스 회귀를 구현함에 있어 우선 소프트맥스 함수의 비용 함수를 low-level로 구현해봅시다. 3개의 원소를 가진 벡터 텐서를 정의하고, 이 텐서를 통해 소프트맥스 함수를 이해해보겠습니다.

In [2]:
z = torch.FloatTensor([1, 2, 3])
hypothesis = F.softmax(z, dim=0)
hypothesis

tensor([0.0900, 0.2447, 0.6652])

3개의 원소들이 0과 1 사이의 값을 가지는 벡터로 변환된 것을 알 수 있습니다. 또한 원소 3개의 합은 1임을 알 수 있습니다.

이번에는 비용 함수를 직접 구현하겠습니다. 우선 임의의 $3 \times 5$ 행렬의 크기를 가진 텐서를 만들고 소프트맥스 함수를 적용합니다. 단, 각 샘플에 대해서 소프트맥스 함수를 적용하여야 하므로 두 번째 차원에 대해서 소프트맥스 함수를 적용한다는 의미에서 dim=1을 써줍니다. 다시 말해 행의 합은 1입니다.

In [3]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
hypothesis

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)

이제 행의 원소들의 합은 1이 되는 텐서로 변환되었습니다. 우리가 만든 소프트맥스 함수의 출력값은 결국 예측값입니다. 즉, 위 텐서는 3개의 샘플에 대해서 5개의 클래스 중 어떤 클래스가 정답인지를 예측한 결과입니다.

이제 각 샘플에 대해 임의의 레이블을 만들고 원-핫 인코딩을 수행합니다.

In [4]:
# 샘플에 대해 임의의 레이블 생성
y = torch.randint(5, (3,)).long()
print(y)

# 각 레이블에 대해서 원-핫 인코딩 수행
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([0, 2, 1])


tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

원-핫 인코딩 과정을 살펴보면 먼저 torch.zeros_like(hypothesis)를 통해 모든 원소가 0이고 크기는 hypothesis와 같은 텐서를 만듭니다. y.unsqueeze(1)은 (3,)의 크기를 (3 x 1)의 크기로(전치와 같은 결과) 바꿔줍니다. 그리고 scatter의 첫 인자로 dim=1에 대해 수행하라고 지정, 세번째 인자에 1을 넣음으로 두번째 인자인 y.unsqueeze(1)가 알려주는 위치에 1을 넣어줍니다. 그 결과가 수행된 결과입니다.

이제 비용 함수를 위한 재료들은 모두 정의했습니다. 소프트맥스의 비용 함수는 다음과 같았습니다.

$$cost(W) = - \frac{1}{n} \sum_{i=1}^n \sum_{j=1}^k y_j^{(i)} \log(p_j^{(i)})$$

마이너스를 뒤로 빼도 식은 성립합니다.

$$cost(W) = \frac{1}{n} \sum_{i=1}^n \sum_{j=1}^k y_j^{(i)} \times (-\log(p_j^{(i)}))$$

$\sum_{j=1}^k$는 sum(dim=1)로, $\frac{1}{n}\sum_{i=1}^n$은 mean()으로 구현하여 코드로 나타내면 다음과 같습니다.

In [5]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
cost

tensor(1.4689, grad_fn=<MeanBackward0>)

# **2. 파이토치로 소프트맥스의 비용 함수 구현하기 (high-level)**
---

이제 소프트맥스의 비용 함수를 더 하이 레벨로 구현해보겠습니다.

## 2.1 F.softmax() + torch.log() = F.log_softmax()

앞서 소프트맥스 함수의 결과에 로그를 씌울 때는 소프트맥스 함수의 출력값을 로그 함수의 입력으로 사용했습니다.

In [6]:
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

그러나 파이토치에선 두 개의 함수를 결합한 F.log_softmax()함수가 존재한다.

In [7]:
torch.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

둘의 결과가 같음을 확인할 수 있습니다.

## 2.2 F.log_softmax() + F.nll_loss() = F.cross_entropy()

앞서 구한 비용 함수는 다음과 같았습니다.

In [8]:
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

그런데 앞서 배운 F.log_softmax()를 활용하여 torch.log(F.softmax(z, dim=1))을 더욱 간단하게 할 수 있다.

In [9]:
(y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

이를 더 간단하게 하는 방법은 F.nll_loss()를 사용해 바로 실제값을 인자로 사용하는 것이다.

In [10]:
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

여기서 nll이란 Negative Log Likehood의 약자입니다. 이를 더 간단하게 나타낼 수 있습니다.

In [11]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

F.cross_entropy는 비용 함수에 소프트맥스 함수까지 포함하고 있습니다. 